In [5]:
options(scipen=999) # Removes scientific notation (useful for when we want readable plots)
options(repr.plot.width=25, repr.plot.height=15) # Make plots larger in Jupyter output
library(tidyverse) #The mother of all analysis packages
library(rvest) #Scraping web pages
library(httr) #Lets us use requests for data from APIs
library(tidyr) #Functions to help keep our data tidy
library(magrittr) #Piping
library(purrr) #Useful functions for general df wrangling, especially for functions
library(glue) #Gives us useful functions for strings
library(stringr) #More useful functions for strings
library(polite) #For web scraping
library(xml2) #Allows us to work with xml files obtained through scraping
library(GGally) #Produces handy scatterplot matrix with correlations between columns in a dataframe
library(leaps) #Contains regsubsets() to help us select predictors for our model
library(car) #Contains AIC, which we're using for model analysis
library("jsonlite") #Gives functions to help us work with json from API request content
library(plotly) #Allows us to develop interactive graphs - compatible with ggplot2
library(ggthemr) #Extra nice-looking themes for ggplot2 graphs
library(tm) #Contains the function which allows us to remove numbers from a string
ggthemr('dust')

In [6]:
codes_url = 'https://www.worlddata.info/countrycodes.php'
code_page = read_html(codes_url) #Reads in the html body from the webpage
code_page %>% glimpse() #Double check that the body has read in correctly (we look at the classes of the objects)

countries_url = 'http://www.energybc.ca/cache/nuclear/nuclear2/www.oecd.org/document/1/0,2340,en_2649_201185_1889402_1_1_1_1,00.html'
countries_page = read_html(countries_url) #Reads in the html body from the webpage
countries_page %>% glimpse() #Double check that the body has read in correctly (we look at the classes of the objects)

List of 2
 $ node:<externalptr> 
 $ doc :<externalptr> 
 - attr(*, "class")= chr [1:2] "xml_document" "xml_node"
List of 2
 $ node:<externalptr> 
 $ doc :<externalptr> 
 - attr(*, "class")= chr [1:2] "xml_document" "xml_node"


In [7]:
oecd_countries = countries_page %>% html_nodes(xpath = '//*[@class="more"]') %>% html_text()
oecd_countries = oecd_countries[1:length(oecd_countries)-1]
#In the data that we read in, South Korea was called Korea - which is different to what it's called in the OECD dataset,
#thus we need to change them in order to appropriately merge this with the other scraped OECD data
for (i in seq.int(1, length(oecd_countries))){
    if (oecd_countries[i] == 'Korea'){ #For each country in the list of OECD countries, if the country is 'Korea' change its
                                        #value to 'South Korea'
        oecd_countries[i] = 'South Korea'
    }
}

In [8]:
all_codes = code_page %>% html_nodes(xpath = '//*[@class="std100 hover"]') %>% html_text()
#Using xpath selector to extract the desired text from page of country codes
country_codes = unlist(strsplit(all_codes, "[.]"))
#After extracting using the xpath selector, the codes had "." in between them, this function removes these and returns a list
#We then use "unlist()" to turn it into an array

output = array()
countries = array()
codes = array()
#Initialising variables
for (country_data in country_codes){
    country_data = substring(country_data, 1, nchar(country_data)-8)
    country_data = substring(country_data, 3, nchar(country_data))
    country_data = removeNumbers(country_data)
    code = substring(country_data, nchar(country_data)-2, nchar(country_data)) 
    country = substring(country_data, 1, nchar(country_data)-3)
    country = substring(country, 1, nchar(country)-2)
    countries = countries %>% append(country)
    codes = codes %>% append(code)
}
#Essentially, this data was really messy to read in (there were multiple different values for each country, such as website suffixes and mobile codes - all of different lengths)
#so I applied a few calls to the substring function to isolate the country name and the code we were interested in (the 3 letter one) from the other unwanted data.
#The for loop adds the countries to the countries array and codes to the codes array

#Note we are including non-OECD countries here - which will get weeded out when comparing this array to the array of OECD countries

output = output[4:length(output)]
countries = countries[5:length(countries)-1]
codes = codes[5:length(codes)-1]
#We remove some non-useful data here, containing irrelevant non-OECD countries
countries
codes
#Double checking our arrays are accurate

[1] "Albania"                                  
  [2] "Algeria"                                  
  [3] "American Samoa"                           
  [4] "Andorra"                                  
  [5] "Angola"                                   
  [6] "Anguilla"                                 
  [7] "Antarct"                                  
  [8] "Antigua and Barbuda"                      
  [9] "Argentina"                                
 [10] "Armenia"                                  
 [11] "Aruba"                                    
 [12] "Australia"                                
 [13] "Austria"                                  
 [14] "Azerbaijan"                               
 [15] "Bahamas"                                  
 [16] "Bahrain"                                  
 [17] "Bangladesh"                               
 [18] "Barbados"                                 
 [19] "Belarus"                                  
 [20] "Belgium"                                  
 [21] "Belize"                                   
 [22] "Benin"                                    
 [23] "Bermuda"                                  
 [24] "Bhutan"                                   
 [25] "Bolivia"                                  
 [26] "Bonaire, Saint Eustatius and S"           
 [27] "Bosnia and Herzegovina"                   
 [28] "Botswana"                                 
 [29] "Bouvet Isl"                               
 [30] "Brazil"                                   
 [31] "British Indian Ocean Territ"              
 [32] "British Virgin Islands"                   
 [33] "Brunei"                                   
 [34] "Bulgaria"                                 
 [35] "Burkina Faso"                             
 [36] "Burma"                                    
 [37] "Burundi"                                  
 [38] "Cambodia"                                 
 [39] "Cameroon"                                 
 [40] "Canada"                                   
 [41] "Cape Verde"                               
 [42] "Cayman Islands"                           
 [43] "Central African Republic"                 
 [44] "Chad"                                     
 [45] "Chile"                                    
 [46] "China"                                    
 [47] "Christmas Isl"                            
 [48] "Cocos (Keeling) Isla"                     
 [49] "Colombia"                                 
 [50] "Comoros"                                  
 [51] "Cook Islands"                             
 [52] "Costa Rica"                               
 [53] "Croatia"                                  
 [54] "Cuba"                                     
 [55] "Cura"                                     
 [56] "Cyprus"                                   
 [57] "Czechia"                                  
 [58] "Democratic Republic of the Congo"         
 [59] "Denmark"                                  
 [60] "Djibouti"                                 
 [61] "Dominica"                                 
 [62] "Dominican Republic"                       
 [63] "East Timor"                               
 [64] "Ecuador"                                  
 [65] "Egypt"                                    
 [66] "El Salvador"                              
 [67] "Equatorial Guinea"                        
 [68] "Eritrea"                                  
 [69] "Estonia"                                  
 [70] "Eswatini"                                 
 [71] "Ethiopia"                                 
 [72] "Falkland Isla"                            
 [73] "Faroe Island"                             
 [74] "Federated States of Micronesia"           
 [75] "Fiji"                                     
 [76] "Finland"                                  
 [77] "France"                                   
 [78] "French Gui"                               
 [79] "French Polyne"                            
 [80] "French Southern and Antarctic La"         
 [

[1] "ALB" "DZA" "ASM" "AND" "AGO" "AIA" "aAQ" "ATG" "ARG" "ARM" "ABW" "AUS"
 [13] "AUT" "AZE" "BHS" "BHR" "BGD" "BRB" "BLR" "BEL" "BLZ" "BEN" "BMU" "BTN"
 [25] "BOL" "aBQ" "BIH" "BWA" "dBV" "BRA" "yIO" "VGB" "BRN" "BGR" "BFA" "MMR"
 [37] "BDI" "KHM" "CMR" "CAN" "CPV" "CYM" "CAF" "TCD" "CHL" "CHN" "dCX" "sCC"
 [49] "COL" "COM" "COK" "CRI" "HRV" "CUB" "oCW" "CYP" "CZE" "COD" "DNK" "DJI"
 [61] "DMA" "DOM" "TLS" "ECU" "EGY" "SLV" "GNQ" "ERI" "EST" "SWZ" "ETH" "sFK"
 [73] "OFR" "FSM" "FJI" "FIN" "FRA" "aGF" "aPF" "sTF" "GAB" "GMB" "GEO" "DEU"
 [85] "GHA" "GIB" "GRC" "LGR" "GRD" "eGP" "GUM" "GTM" "GGY" "GIN" "GNB" "GUY"
 [97] "HTI" "sHM" "HND" "HKG" "HUN" "ISL" "IND" "IDN" "IRN" "IRQ" "IRL" "Man"
[109] "ISR" "ITA" "CIV" "JAM" "JPN" "JEY" "JOR" "KAZ" "KEN" "KIR" "XKX" "KWT"
[121] "KGZ" "LAO" "LVA" "LBN" "LSO" "LBR" "LBY" "LIE" "LTU" "LUX" "oMO" "MDG"
[133] "MWI" "MYS" "MDV" "MLI" "MLT" "MHL" "eMQ" "MRT" "MUS" "eYT" "MEX" "MDA"
[145] "MNG" "MNE" "tMS" "MAR" "MOZ" "NAM" "NRU" "NPL" "NLD" "NCL" "NZL" "NIC"
[157] "NER" "NGA" "eNU" "dNF" "PRK" "MKD" "sMP" "NOR" "OMN" "PAK" "PLW" "PSE"
[169] "PAN" "PNG" "PRY" "PER" "PHL" "sPN" "POL" "PRT" "MCO" "PRI" "QAT" "COG"
[181] "nRE" "ROU" "RUS" "RWA" "yBL" "aSH" "KNA" "LCA" "MFM" "nPM" "VCT" "WSM"
[193] "SMR" "STP" "SAU" "SEN" "SRB" "SYC" "SLE" "SGP" "nSX" "SVK" "SVN" "SLB"
[205] "SOM" "ZAF" "sGS" "KOR" "SSS" "ESP" "LKA" "SDN" "SUR" "dSJ" "SWE" "CHE"
[217] "SYR" "TWN" "TJK" "TZA" "THA" "TGO" "uTK" "TON" "TTO" "TUN" "TUR" "TKM"
[229] "sTC" "TUV" "UGA" "UKR" "ARE" "GBR" "USA" "URY" "UZB" "VUT" "VAV" "VEN"
[241] "VNM" "VIR" "aWF" "ESH" "YEM" "ZMB" "ZWE"

In [10]:
#The following function which returns a list which maps a country to a code, thus when converting countries into codes can be used like:
#country = list_for_conversion[code], thus we can convert the codes into countries to be used as the primary key (along with year of observation) when joining the table
#containing labour hours to the main table containing all other data.
countriesAndCodes = function(countries, codes) {
    output_countries = array() #initializes countries array
    output_codes = array() #initializes codes array
    for (i in seq.int(1, length(countries))){ #Both arrays are the same length
        if (countries[i] %in% oecd_countries){ #Checks if country is in the OECD
            output_countries = output_countries %>% append(countries[i]) #Appends the country to the countries array
            output_codes = output_codes %>% append(codes[i]) #Appends the code to the codes array
        }
    }
    output_list = as.list(output_countries[2:length(output_countries)]) #Creates the list, with the countries as values
    names(output_list) = as.list(output_codes[2:length(output_codes)]) #Sets codes to be the keys to the countries
    return(output_list) #Returns the list
}

list_for_conversions = countriesAndCodes(countries, codes) #Applies the function
#Now to get the name of a country using the function output - just use country_name = as.character(list_for_conversions['country_code'])
#Example

united_states = list_for_conversions['USA']
united_states
#Now we can change codes into country names!

$AUS
[1] "Australia"

$AUT
[1] "Austria"

$BEL
[1] "Belgium"

$CAN
[1] "Canada"

$CHL
[1] "Chile"

$DNK
[1] "Denmark"

$EST
[1] "Estonia"

$FIN
[1] "Finland"

$FRA
[1] "France"

$DEU
[1] "Germany"

$GRC
[1] "Greece"

$HUN
[1] "Hungary"

$ISL
[1] "Iceland"

$IRL
[1] "Ireland"

$ISR
[1] "Israel"

$ITA
[1] "Italy"

$JPN
[1] "Japan"

$LUX
[1] "Luxembourg"

$MEX
[1] "Mexico"

$NLD
[1] "Netherlands"

$NZL
[1] "New Zealand"

$NOR
[1] "Norway"

$POL
[1] "Poland"

$PRT
[1] "Portugal"

$SVN
[1] "Slovenia"

$KOR
[1] "South Korea"

$ESP
[1] "Spain"

$SWE
[1] "Sweden"

$CHE
[1] "Switzerland"

$TUR
[1] "Turkey"

$GBR
[1] "United Kingdom"

$USA
[1] "United States"